# **Spam Classification**

The aim of this project is to bulid a spam classifier using Naive Bayes Classification with Laplace Smoothening

In [63]:
#importing the necessary packages
import numpy as np
import random

In [64]:
#importing the data
X=np.genfromtxt('.//spam_X.csv',dtype='str',delimiter='\n',skip_header=0,invalid_raise=True)
Y=np.genfromtxt('.//spam_Y.csv',dtype='str',delimiter='\n')
Y=np.where(Y=='1','0','1')

In [65]:
print(X.shape)

(4296,)


The Dataset consists of 4296 Emails

Class '0' denotes 'spam' Emails

Class '1' denotes 'non-spam' Emails

In [66]:
print("Before preprocessing the feature sentences are")
print()
print(X[0])
print()
print(X[200])
print()
print(X[4000])
print()

Before preprocessing the feature sentences are

"Subject: re : faculty lunch  alison ,  i recommended inviting duane seppi and steven shreve . i would also invite  brian routledge .  i don ' t know him but heard many good things about him . kevin kindall may have  other recommendations .  vince  enron north america corp .  from : mary alison bailey 09 / 08 / 2000 05 : 04 pm  to : shirley crenshaw / hou / ect @ ect , vince j kaminski / hou / ect @ ect , kevin  kuykendall / hou / ect @ ect , kevin kuykendall / hou / ect @ ect  cc : kristin gandy / na / enron @ enron  subject : faculty lunch  kristin had said she was interested in a faculty lunch , and kevin said he  would host it . are there any professors you would recommend be invited ?  here is a list of finance faculty :  robert dammon rdl 9 @ andrew . cmu . edu 412 . 268 . 3696  richard green rgob @ andrew . cmu . edu 412 . 268 . 2302  david heath heath @ andrew . cmu . edu 412 . 268 . 2545  christine parlour parlourc @ andrew . cmu

In [67]:
#preprocessing the feature sentence to remove numbers and special characters
#also converting all upper case letters to lower case
for i in range(0,X.shape[0],1):
    s=X[i]
    flag=0
    s1=""
    for j in range(0,len(s),1):
        if(ord(s[j])>=65 and ord(s[j])<=90):
            if(flag==1 and len(s1)>0):
                s1=s1+" "
            s1=s1+chr(ord(s[j])+32)
            flag=0
        elif(ord(s[j])>=97 and ord(s[j])<=122):
            if(flag==1 and len(s1)>0):
                s1=s1+" "
            s1=s1+s[j]
            flag=0
        elif(s[j]==' '):
            flag=1
        elif(s[j]=="'"):
            continue
        else:
            flag=1
    X[i]=s1

In [68]:
print("After preprocessing the feature sentences are")
print()
print(X[0])
print()
print(X[200])
print()
print(X[4000])
print()

After preprocessing the feature sentences are

subject re faculty lunch alison i recommended inviting duane seppi and steven shreve i would also invite brian routledge i don t know him but heard many good things about him kevin kindall may have other recommendations vince enron north america corp from mary alison bailey pm to shirley crenshaw hou ect ect vince j kaminski hou ect ect kevin kuykendall hou ect ect kevin kuykendall hou ect ect cc kristin gandy na enron enron subject faculty lunch kristin had said she was interested in a faculty lunch and kevin said he would host it are there any professors you would recommend be invited here is a list of finance faculty robert dammon rdl andrew cmu edu richard green rgob andrew cmu edu david heath heath andrew cmu edu christine parlour parlourc andrew cmu edu brian routledge rout andrew cmu edu duane seppi ds andrew cmu edu steven shreve shreve andrew cmu edu chester spatt cs z andrew cmu edu christopher telmer telmerc andrew cmu edu stanl

In [69]:
#splitting the data into training data and testing data
train_X=X[:int(X.shape[0]*0.8)]
train_Y=Y[:int(Y.shape[0]*0.8)]
test_X=X[int(X.shape[0]*0.8):]
test_Y=Y[int(Y.shape[0]*0.8):]
m=train_X.shape[0]

In [72]:
#Using 10 fold cross validation to tune the hyper parameter alpha
alpha=[]
for jj in range(0,10,1):
    print("Cross validation Phase",jj+1)
    val_x=train_X[int(jj*m*0.1):int((jj+1)*m*0.1)]
    val_y=train_Y[int(jj*m*0.1):int((jj+1)*m*0.1)]
    train_x=np.hstack((train_X[:int(jj*m*0.1)],train_X[int((jj+1)*m*0.1):]))
    train_y=np.hstack((train_Y[:int(jj*m*0.1)],train_Y[int((jj+1)*m*0.1):]))
    #using the training data to form a class wise dictionary containing words that appear in a particular class as keys and their frequencies as values
    d={}
    for i in range(0,train_x.shape[0],1):
        if(train_y[i] in d.keys()):
            for j in train_x[i].split(" "):
                if(j in d[train_y[i]].keys()):
                    d[train_y[i]][j]+=1
                else:
                    d[train_y[i]][j]=1
        else:
            d[train_y[i]]={}
            for j in train_x[i].split(" "):
                if(j in d[train_y[i]].keys()):
                    d[train_y[i]][j]+=1
                else:
                    d[train_y[i]][j]=1
    class_wise_words_dict=d
    #using the training data to compute the prior probability of each class
    d={}
    u=np.unique(train_y,return_counts=True)
    s=np.sum(u[1])
    for i in range(0,len(u[0]),1):
        d[u[0][i]]=u[1][i]/s
    prior_prob=d
    #using the validation data to tune the hyperparameter alpha
    accuracy=0
    a=[1,3,5,10,30,50,100,300,500]
    classes=np.unique(train_y)
    for alph in a:
        #computing the denominator of the liklihood for each class
        print("For alpha =",alph)
        denominator_d={}
        t_s=0
        for i in class_wise_words_dict:
            s=0
            for j in class_wise_words_dict[i]:
                s=s+class_wise_words_dict[i][j]
                t_s=t_s+1
            denominator_d[i]=s
        for i in denominator_d:
            denominator_d[i]=denominator_d[i]+alph*t_s
        class_wise_denominators_dict=denominator_d
        #predicting the class for the validation data using naive bayes algorithm
        y_pred=[]
        for i in range(0,val_x.shape[0],1):
            prob=0
            for c in classes:
                p=0
                for j in val_x[i].split(" "):
                    if(j in class_wise_words_dict[c].keys()):
                        p=p+np.log((class_wise_words_dict[c][j]+alph)/class_wise_denominators_dict[c])
                    else:
                        p=p+np.log(alph/class_wise_denominators_dict[c])
                p=p+np.log(prior_prob[c])
                if(p>prob or prob==0):
                    prob=p
                    cl=c
            y_pred=y_pred+[cl]
        #evaluating the model using the weighted F1_score
        f1=0
        c_m=np.zeros((len(classes),len(classes)),dtype='int64')
        for i in range(0,len(y_pred),1):
            c_m[int(y_pred[i]),int(test_Y[i])]+=1
        tp=np.zeros(len(classes),dtype='int64')
        fp=np.zeros(len(classes),dtype='int64')
        fn=np.zeros(len(classes),dtype='int64')
        w=np.zeros(len(classes),dtype='int64')
        pre=np.zeros(len(classes),dtype='float64')
        rec=np.zeros(len(classes),dtype='float64')
        F1=np.zeros(len(classes),dtype='float64')
        F1_score=0
        for i in classes:
            i=int(i)
            tp[i]=c_m[i,i]
            fp[i]=np.sum(c_m[i,:])-tp[i]
            fn[i]=np.sum(c_m[:,i])-tp[i]
            w[i]=np.sum(c_m[:,i])
            pre[i]=tp[i]/(tp[i]+fp[i])
            rec[i]=tp[i]/(tp[i]+fn[i])
            F1[i]=2*pre[i]*rec[i]/(pre[i]+rec[i])
            F1_score=F1_score+F1[i]*w[i]
        F1_score=F1_score/np.sum(w)
        print("F1_score =",F1_score)
        if(F1_score>f1):
            f1=F1_score
            Alpha=alph
    alpha+=[Alpha]

Cross validation Phase 1
For alpha = 1
F1_score = 0.6362735936162235
For alpha = 3
F1_score = 0.6394132752687111
For alpha = 5
F1_score = 0.6411832338569231
For alpha = 10
F1_score = 0.6472267615502028
For alpha = 30
F1_score = 0.6210224083881212
For alpha = 50


C:\Users\Aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:88: RuntimeWarning: invalid value encountered in double_scalars


F1_score = nan
For alpha = 100
F1_score = nan
For alpha = 300
F1_score = nan
For alpha = 500


C:\Users\Aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in longlong_scalars


F1_score = nan
Cross validation Phase 2
For alpha = 1
F1_score = 0.6177830684693849
For alpha = 3
F1_score = 0.6196392253957026
For alpha = 5
F1_score = 0.6280316374744345
For alpha = 10
F1_score = 0.6364865330572169
For alpha = 30
F1_score = nan
For alpha = 50
F1_score = nan
For alpha = 100
F1_score = nan
For alpha = 300
F1_score = nan
For alpha = 500
F1_score = nan
Cross validation Phase 3
For alpha = 1
F1_score = 0.6501457725947521
For alpha = 3
F1_score = 0.6536465661830093
For alpha = 5
F1_score = 0.6478472986733134
For alpha = 10
F1_score = 0.6580010095911156
For alpha = 30
F1_score = 0.6382658019476013
For alpha = 50
F1_score = nan
For alpha = 100
F1_score = nan
For alpha = 300
F1_score = nan
For alpha = 500
F1_score = nan
Cross validation Phase 4
For alpha = 1
F1_score = 0.6334633020571286
For alpha = 3
F1_score = 0.6402147288673135
For alpha = 5
F1_score = 0.6381455198358413
For alpha = 10
F1_score = 0.625818559124188
For alpha = 30
F1_score = 0.6295779833632381
For alpha = 50

In [73]:
Alpha=np.unique(alpha,return_counts=True)
alpha=Alpha[0][np.argmax(Alpha[1])]
print("alpha =",alpha)

alpha = 10


In [74]:
#training the model using the training and validation data
#forming a class wise dictionary containing words that appear in a particular class as keys and their frequencies as values
d={}
for i in range(0,train_X.shape[0],1):
    if(train_Y[i] in d.keys()):
        for j in train_X[i].split(" "):
            if(j in d[train_Y[i]].keys()):
                d[train_Y[i]][j]+=1
            else:
                d[train_Y[i]][j]=1
    else:
        d[train_Y[i]]={}
        for j in train_X[i].split(" "):
            if(j in d[train_Y[i]].keys()):
                d[train_Y[i]][j]+=1
            else:
                d[train_Y[i]][j]=1
class_wise_words_dict=d
#computing the prior probability of each class
d={}
u=np.unique(train_Y,return_counts=True)
s=np.sum(u[1])
for i in range(0,len(u[0]),1):
    d[u[0][i]]=u[1][i]/s
prior_prob=d
#computing the denominator of the liklihood for each class
denominator_d={}
t_s=0
for i in class_wise_words_dict:
    s=0
    for j in class_wise_words_dict[i]:
        s=s+class_wise_words_dict[i][j]
        t_s=t_s+1
    denominator_d[i]=s
for i in denominator_d:
    denominator_d[i]=denominator_d[i]+alpha*t_s
class_wise_denominators_dict=denominator_d

In [75]:
#using the model to predict the labels of the test data
y_pred=[]
for i in range(0,test_X.shape[0],1):
    prob=0
    for c in classes:
        p=0
        for j in test_X[i].split(" "):
            if(j in class_wise_words_dict[c].keys()):
                p=p+np.log((class_wise_words_dict[c][j]+alpha)/class_wise_denominators_dict[c])
            else:
                p=p+np.log(alpha/class_wise_denominators_dict[c])
        p=p+np.log(prior_prob[c])
        if(p>prob or prob==0):
            prob=p
            cl=c
    y_pred=y_pred+[cl]

In [76]:
#generating the confusion matrix
c_m=np.zeros((len(classes),len(classes)),dtype='int64')
for i in range(0,len(y_pred),1):
    c_m[int(y_pred[i]),int(test_Y[i])]+=1

In [78]:
print("The Confusion Matrix is")
print(c_m)

The Confusion Matrix is
[[ 77   2]
 [139 642]]


In [79]:
#computing the weighted F1_score of the model
tp=np.zeros(len(classes))
fp=np.zeros(len(classes))
fn=np.zeros(len(classes))
w=np.zeros(len(classes))
pre=np.zeros(len(classes))
rec=np.zeros(len(classes))
F1=np.zeros(len(classes))
F1_score=0
for i in classes:
    i=int(i)
    tp[i]=c_m[i,i]
    fp[i]=np.sum(c_m[i,:])-tp[i]
    fn[i]=np.sum(c_m[:,i])-tp[i]
    w[i]=np.sum(c_m[:,i])
    pre[i]=tp[i]/(tp[i]+fp[i])
    rec[i]=tp[i]/(tp[i]+fn[i])
    F1[i]=2*pre[i]*rec[i]/(pre[i]+rec[i])
    F1_score=F1_score+F1[i]*w[i]
F1_score=F1_score/np.sum(w)

In [80]:
print("F1_score =",F1_score)

F1_score = 0.8058572288031866


In [81]:
#Calculating Accuracy
accuracy=(tp[0]+tp[1])/test_X.shape[0]
print("accuracy = ",accuracy)

accuracy =  0.836046511627907
